In [1]:
import sympy as sp
import sympy.physics.units as u
from IPython.display import Markdown, Image
from sympy import exp as e
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

sp.printing.str.StrPrinter._default_settings['abbrev'] = True

In [2]:
_AGE     = 27
_WGT     = 61
_CRCL    = 109
_SEX     = "M"

In [16]:
SYMBOLS = sp.symbols(" V_d k_e D C_{max} C_{min} Cl_{Cr} t' \\tau W")

VD , KE , D , CMAX , CMIN, CRCL, TPRIME, TAU, W = SYMBOLS


SYMBOL_DESCRIPTIONS = {
   W :   "Weight (kg)",
   VD :  "Volume of Distibution",
   KE :   "Elimination constant",
   D  :   "Dose",
   CMAX : "Peak Concentration",
   CMIN : "Trough Concentration",
   CRCL : "Creatinine Clearance",
   TPRIME:"Infusion Time (hr)",
   TAU  : "Interval (hr)"
}

EQUATION_ELIM_CONST = sp.Eq( KE,# == 
                                    0.00083 * CRCL + 0.0044)
EQUATION_CALC_CMAX = sp.Eq (CMAX ,
                                    ((D / TPRIME)/ (VD*KE))*((1-e(-KE*TPRIME))/(1-e(-KE*TAU))))
EQUATION_CALC_CMIN = sp.Eq (CMIN,# == 
                                      CMAX * sp.exp(-KE * (-TPRIME + TAU)))
EQUATION_VOL_DISTR = sp.Eq (VD,# ==
                                    0.7 * W)

ALL_EQUATIONS = {
         "VD Equation" : EQUATION_VOL_DISTR,
         "Ke Equation" : EQUATION_ELIM_CONST,
         "CMax Equation" : EQUATION_CALC_CMAX,
         "CMIN Equation" : EQUATION_CALC_CMIN
}

for eq in ALL_EQUATIONS:
   display(eq)

def equationdesc (List):
   head = "|  Eq | Symbols |\n|----|----|\n"
   basestr = "| "
   for eq in List.values() :
      basestr += eq._repr_latex_()
      basestr += " | "
      str2 = ""
      for var in list(eq.rhs.free_symbols) :
         str2 += var._repr_latex_()
         str2 += f" `{SYMBOL_DESCRIPTIONS[var]}` "
      basestr += str2
      basestr += " |\n"
   tablestr = head + basestr
   return tablestr

EQUATION_CALC_CMIN.__annotations__

'VD Equation'

'Ke Equation'

'CMax Equation'

'CMIN Equation'

{'_args': 'Tuple[Basic, ...]', '_mhash': 'Any', 'kind': sympy.core.kind.Kind}

In [10]:
md = equationdesc(ALL_EQUATIONS)
Markdown(md)

|  Eq | Symbols |
|----|----|
| $\displaystyle V_{d} = 0.7 W$ | $\displaystyle W$ `Weight (kg)`  |
$\displaystyle k_{e} = 0.00083 Cl_{Cr} + 0.0044$ | $\displaystyle Cl_{Cr}$ `Creatinine Clearance`  |
$\displaystyle C_{max} = \frac{D \left(1 - e^{- k_{e} t'}\right)}{V_{d} k_{e} t' \left(1 - e^{- \tau k_{e}}\right)}$ | $\displaystyle D$ `Dose` $\displaystyle V_{d}$ `Volume of Distibution` $\displaystyle k_{e}$ `Elimination constant` $\displaystyle \tau$ `Interval (hr)` $\displaystyle t'$ `Infusion Time (hr)`  |
$\displaystyle C_{min} = C_{max} e^{- k_{e} \left(\tau - t'\right)}$ | $\displaystyle k_{e}$ `Elimination constant` $\displaystyle C_{max}$ `Peak Concentration` $\displaystyle \tau$ `Interval (hr)` $\displaystyle t'$ `Infusion Time (hr)`  |


In [11]:
def mdTableSymbols_from_dict (Dict):
   view  = "| SYMBOL | REPR |\n"
   view += "|-------|-------|" 
   for f , d in Dict.items():
      view += "\n| " + f._repr_latex_() + " | " + str(d) + " |"
   display(Markdown(view))
   
mdTableSymbols_from_dict (SYMBOL_DESCRIPTIONS)

| SYMBOL | REPR |
|-------|-------|
| $\displaystyle W$ | Weight (kg) |
| $\displaystyle V_{d}$ | Volume of Distibution |
| $\displaystyle k_{e}$ | Elimination constant |
| $\displaystyle D$ | Dose |
| $\displaystyle C_{max}$ | Peak Concentration |
| $\displaystyle C_{min}$ | Trough Concentration |
| $\displaystyle Cl_{Cr}$ | Creatinine Clearance |
| $\displaystyle t'$ | Infusion Time (hr) |
| $\displaystyle \tau$ | Interval (hr) |

In [ ]:
EQUATION_CALC_CMAX.subs({
            VD : EQUATION_VOL_DISTR.rhs
})

Eq(C_{max}, 1.42857142857143*D*(1 - exp(-k_e*t'))/(W*k_e*t'*(1 - exp(-\tau*k_e))))

In [ ]:
EQUATION_CALC_CMAX.subs({
            TPRIME : 2.6,
            TAU : 8,
            D : 1000,
            KE : EQUATION_ELIM_CONST.rhs.subs({CRCL:99}) ,
            VD : EQUATION_VOL_DISTR.rhs.subs({W : 60})
})

Eq(C_{max}, 42.6654470086362)